# 基本语法

最经典的： 提示词模版 | 大模型 | 输出

chain.invoke({"key", "value"}) 去触发， 并可以填充模版中的变量

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_template("我 爱你 {name}")

model = ChatOpenAI()

output_parser = StrOutputParser()

chain = prompt | model | output_parser            # 加了数据解析器， 是AIMessage对象： <class 'langchain_core.messages.ai.AIMessage'>
chain2 = prompt | model                           # 没有加数据解析器， 是字符串： <class 'str'>

print(type(chain2.invoke({"name": "小明"})))
print(type(chain.invoke({"name": "小明"})))

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>


## 主要组件

### Prompt Templates
用于构造提示词模板：

In [6]:

# 简单模板
prompt = ChatPromptTemplate.from_template("翻译以下文本：{text}")

# 多消息模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的翻译助手"),
    ("user", "翻译：{text}")
])
model = ChatOpenAI()

chain = prompt | model
response = chain.invoke({"text": "Hello, world!"})
print(response)

content='你好，世界！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 35, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Baar6AGIR0BAL9gP9bORGgWJ2e64d', 'finish_reason': 'stop', 'logprobs': None} id='run-a88d7660-339f-4542-b394-40f99d19820e-0' usage_metadata={'input_tokens': 35, 'output_tokens': 7, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### Language Models
支持各种语言模型：

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")


###  Output Parsers

解析模型输出：

In [8]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

# 字符串解析器
str_parser = StrOutputParser()

# JSON解析器
json_parser = JsonOutputParser()

# 高级特性
### 1. 并行执行
使用 RunnableParallel 实现并行处理：

In [9]:
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel({
    "summary": prompt1 | model | StrOutputParser(),
    "analysis": prompt2 | model | StrOutputParser()
})

NameError: name 'prompt1' is not defined

### 2. 条件分支
使用 RunnableBranch 实现条件逻辑：

In [ ]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "question" in x, question_chain),
    (lambda x: "summary" in x, summary_chain),
    default_chain
)

### 3. Lambda函数
在链条中插入自定义逻辑：

In [ ]:
from langchain_core.runnables import RunnableLambda

def custom_processor(input_dict):
    # 自定义处理逻辑
    return processed_data

chain = (
    prompt 
    | model 
    | RunnableLambda(custom_processor)
    | output_parser
)

### 实际应用示例
#### 1. 简单翻译链

In [ ]:
translation_chain = (
    ChatPromptTemplate.from_template("将以下中文翻译成英文：{chinese_text}")
    | ChatOpenAI()
    | StrOutputParser()
)

result = translation_chain.invoke({"chinese_text": "你好世界"})

### RAG（检索增强生成）链

In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(
        "基于以下上下文回答问题：\n\n{context}\n\n问题：{question}"
    )
    | model
    | StrOutputParser()
)

### 3. 多步骤分析链

In [ ]:
analysis_chain = (
    # 首先保存原始输入
    RunnableParallel(
        sentiment_analysis=ChatPromptTemplate.from_template("分析以下文本的情感：{text}") | model | StrOutputParser(),
        original_text=RunnablePassthrough()
    )
    # 现在有了 {"sentiment_analysis": "积极", "original_text": {"text": "我很开心"}}
    
    | RunnableLambda(lambda x: {
        "sentiment": x["sentiment_analysis"], 
        "original_text": x["original_text"]["text"]
    })
    
    | ChatPromptTemplate.from_template(
        "基于情感分析结果：{sentiment}，为原文本提供改进建议：{original_text}"
    )
    | model
    | StrOutputParser()
)

### 异步支持
LCEL 支持异步操作：

In [ ]:
import asyncio

async def run_async_chain():
    result = await chain.ainvoke({"topic": "programming"})
    return result

# 批处理
async def batch_process():
    inputs = [{"topic": "AI"}, {"topic": "ML"}, {"topic": "DL"}]
    results = await chain.abatch(inputs)
    return results

### 流式处理
支持流式输出：

In [ ]:
# 同步流
for chunk in chain.stream({"topic": "technology"}):
    print(chunk, end="", flush=True)

# 异步流
async for chunk in chain.astream({"topic": "technology"}):
    print(chunk, end="", flush=True)